In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
import os

# Chemin du fichier zip dans Google Drive
zip_path = '/content/drive/My Drive/models.zip'

# Décompressez ici
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/models')

# Vérifiez si le fichier est décompressé
if os.path.exists('/content/models/unet.hdf5'):
    print('Fichier unet.hdf5 extrait avec succès.')

In [ ]:
import tensorflow as tf

model_path = "/content/models/models/unet.hdf5"
model = tf.keras.models.load_model(model_path, custom_objects={'dice_loss': dice_loss,'dice_coef': dice_coef,'iou_coef':iou_coef})

In [ ]:
# function to create dice coefficient
def dice_coef(y_true, y_pred, smooth=100):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)

    intersection = K.sum(y_true_flatten * y_pred_flatten)
    union = K.sum(y_true_flatten) + K.sum(y_pred_flatten)
    return (2 * intersection + smooth) / (union + smooth)

# function to create dice loss
def dice_loss(y_true, y_pred, smooth=100):
    return -dice_coef(y_true, y_pred, smooth)

# function to create iou coefficient
def iou_coef(y_true, y_pred, smooth=100):
    intersection = K.sum(y_true * y_pred)
    sum = K.sum(y_true + y_pred)
    iou = (intersection + smooth) / (sum - intersection + smooth)
    return iou

In [ ]:
# Convertir le modèle en TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
# Enregistrer le modèle converti
tflite_model_path = '/content/models/model.tflite'  # Changez ce chemin si nécessaire
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
